This file preserve the SQL query I used for task 1. 

### Q1: How many bike trips have been started in San Jose?

In [ ]:
#standardSQL
"""SELECT
start_station_name, lon, lat, COUNT(trip_id) AS cnt
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips` AS trip
INNER JOIN 
  (SELECT station_id, region_id, lon, lat
  FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_station_info` AS S
  INNER JOIN 
    (SELECT region_id AS id
    FROM`bigquery-public-data.san_francisco_bikeshare.bikeshare_regions`
    WHERE name = "San Jose")AS R
  ON S.region_id = R.id
    ) AS station
ON trip.start_station_id =station.station_id
GROUP BY start_station_name, lon, lat"""



### Q2: Which bike station has the oldest members in average?

In [ ]:
#standardSQL
Q2a = """SELECT
start_station_name AS station, "start" AS station_type, EXTRACT(year FROM start_date) AS year,  AVG(EXTRACT(year FROM start_date)-member_birth_year) AS average_age
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips` 
GROUP BY station, year
UNION DISTINCT
SELECT
end_station_name AS station, "end" AS station_type, EXTRACT(year FROM end_date) AS year,  AVG(EXTRACT(year FROM start_date)-member_birth_year) AS average_age
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
GROUP BY station, year
ORDER BY average_age DESC"""


In [ ]:
#standardSQL
Q2b = """SELECT station, AVG(old_pct) AS mean,  AVG(old_pct) - 1.96 * STDDEV_SAMP(old_pct) AS ci_l, AVG(old_pct) + 1.96 * STDDEV_SAMP(old_pct) AS ci_h 
FROM
  (SELECT   
  station, d, SUM(old)/COUNT(trip_id)*100 AS old_pct
  FROM 
    (SELECT
    start_station_name AS station, EXTRACT(date FROM start_date) AS d, trip_id,
    CASE
      WHEN EXTRACT(year FROM start_date) = 2017 AND member_birth_year < 1953 THEN 1
      WHEN EXTRACT(year FROM start_date) = 2018 AND member_birth_year < 1954 THEN 1
      ELSE 0
    END AS old,
    FROM
      `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
      WHERE  EXTRACT(year FROM start_date) > 2016) 
    GROUP BY station, d)
GROUP BY station"""

In [ ]:
Q2c = """

SELECT station, age, COUNT(trip_id) AS cnt
FROM
    (SELECT
    start_station_name AS station, trip_id, 
    CASE
      WHEN EXTRACT(year FROM start_date) = 2017 THEN 2017 - member_birth_year
      WHEN EXTRACT(year FROM start_date) = 2018 THEN 2018 - member_birth_year
    END AS age
    FROM
      `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
      WHERE  EXTRACT(year FROM start_date) > 2016 AND (start_station_name = "San Carlos St at 11th St" OR start_station_name = "Mission St at 1st St"))
GROUP BY station, age
"""

### Q3: During which month in 2018 did customers ride the longest per trip?

In [ ]:
"""SELECT month, avg(len) AS avg_len
FROM
  (SELECT timestamp_diff(end_date, start_date, second) AS len, EXTRACT(month from  start_date) AS month
  FROM
    `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
  WHERE EXTRACT(year from  start_date) = 2018 AND EXTRACT(month from  start_date) =   EXTRACT(month from  end_date))
GROUP BY month 
ORDER BY avg_len DESC"""

In [ ]:
Q3b = """
SELECT  
    month, duration,COUNT(trip_id) AS cnt
FROM(
    SELECT 
        trip_id,
        EXTRACT(MONTH from start_date) AS month,
        TIMESTAMP_DIFF(end_date, start_date, minute) AS duration,
    FROM 
        `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips` 
    WHERE EXTRACT(YEAR from start_date) = 2018)
GROUP BY month, duration
"""

### Q4: What are the top 3 starting stations for customers with a subscription?

In [ ]:
"""
SELECT station, year, count(trip_id) AS cnt
FROM
  (SELECT start_station_id AS id, 
    FIRST_VALUE(start_station_name) OVER(PARTITION BY start_station_id ORDER BY start_date DESC) AS station, 
    Extract(year from start_date) AS year, 
    trip_id
  FROM
    `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
  WHERE subscriber_type = "Subscriber")
GROUP BY  station, year

"""

### Q5: What is the average time difference between start dates of bike trips for the station “SAP Center”?

In [ ]:
"""
SELECT 
  MIN(minimum) AS minimum,
  MIN(q1) AS q1, 
  MIN(q2) AS q2,
  MIN(q3) AS q3,
  MIN(maximum) AS maximum , 
  AVG(gap) AS mean,

FROM
  (SELECT
  PERCENTILE_CONT(TIMESTAMP_DIFF(start_date, next, second), 0) OVER() AS minimum,
  PERCENTILE_CONT(TIMESTAMP_DIFF(start_date, next, second), 0.25) OVER() AS q1,
  PERCENTILE_CONT(TIMESTAMP_DIFF(start_date, next, second), 0.5) OVER() AS q2,
  PERCENTILE_CONT(TIMESTAMP_DIFF(start_date, next, second), 0.75) OVER() AS q3,
  PERCENTILE_CONT(TIMESTAMP_DIFF(start_date, next, second), 1) OVER() AS maximum,
  TIMESTAMP_DIFF(start_date, next, second) AS gap,
  FROM 
    (SELECT start_date, LAG(start_date)OVER (ORDER BY start_date) AS next
    FROM
    `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips` 
    WHERE start_station_name = "SAP Center"))


"""

In [ ]:
Q5b="""
#standardSQL
SELECT 
    start_date,
    next,
    TIMESTAMP_DIFF(start_date, next, second) AS gap,
FROM 
    (SELECT start_date, LAG(start_date)OVER (ORDER BY start_date) AS next
    FROM
    `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips` 
    WHERE start_station_name = "SAP Center")
"""